In [1]:
import base64
import six
import requests
import pandas as pd

In [2]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 88B8-561F

 Directory of C:\Users\dsilva2\EMAT22110_FA21

11/02/2021  03:12 PM    <DIR>          .
11/02/2021  03:12 PM    <DIR>          ..
11/02/2021  03:08 PM    <DIR>          .ipynb_checkpoints
09/02/2021  04:32 PM             5,319 Aug31_inclass-practice.ipynb
09/30/2021  04:41 PM             1,623 bls_data.csv
09/14/2021  04:27 PM            20,354 chess_buds_messages.csv
10/19/2021  01:17 PM            58,901 COVID-Raffle-data-collection.ipynb
09/21/2021  04:30 PM           313,653 DataFrame-Methods-and-visuals.ipynb
10/28/2021  05:07 PM           118,748 datetime-and-figure-axis.ipynb
09/20/2021  01:46 PM           134,652 emoji-encodings-from-fb-data.ipynb
11/02/2021  03:06 PM            53,423 Equal_Track_Data.csv
09/07/2021  11:16 AM           102,772 Explore_IG-data.ipynb
09/07/2021  02:25 PM    <DIR>          facebook-davidsilva100046_full
09/07/2021  02:10 PM     1,857,380,869 facebook-davidsilva100046_full.zip
09/07

In [3]:
tokens = pd.read_csv('MONTEROApp_Keys-9-23-21.txt', sep = "\t", header = None).T
tokens.columns = tokens.iloc[0]

In [4]:
tokens = tokens[1:]

In [ ]:
tokens

In [ ]:
tokens['Client_ID'].iloc[0]

In [ ]:
tokens['Client_Secret'].iloc[0]

In [8]:
def start_session_headers(client_id, client_secret):
    client_cred = base64.b64encode(str(client_id + ":" + client_secret).encode('ascii'))
    header = {'Authorization': 'Basic %s' % (client_cred.decode('ascii'))}
    return header

In [9]:
def connect_to_client_cred(url, header):
    payload = {'grant_type' : 'client_credentials'}
    response = requests.request("POST", url, headers = header, data = payload)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [10]:
def get_access_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    header = start_session_headers(client_id, client_secret)
    json_response = connect_to_client_cred(url, header)
    return json_response

In [11]:
session_start_data = get_access_token(tokens['Client_ID'].iloc[0], tokens['Client_Secret'].iloc[0])

200


#### Now that we have all the pieces, we can make sensible functions that stitch together the pieces for us
For each API call we need a `header` and an `endpoint`. We are going to use three functions to accomplish this. One will be to create a header, one will be to connect to any arbitrary endpoint, and one will be specify the endpoint. 

In [12]:
def create_header(session_token):
    header = {"Authorization" : "Bearer {}".format(session_token)}
    return header

In [13]:
def connect_to_endpoint(url, header):
    response = requests.request("GET", url, headers = header)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [14]:
def album_features(album_id):
    url = 'https://api.spotify.com/v1/albums/{}'.format(album_id)
    header = create_header(session_start_data['access_token'])
    json_response = connect_to_endpoint(url, header)
    return json_response

In [15]:
montero_data = album_features('6pOiDiuDQqrmo5DbG0ZubR')

200


In [16]:
montero_data.keys()

dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])

In [17]:
equal_data = album_features('32iAEBstCjauDhyKpGjTuq')

200


In [18]:
equal_data.keys()

dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])

In [19]:
def audio_features(track_id):
    url = 'https://api.spotify.com/v1/audio-features/{}'.format(track_id)
    header = create_header(session_start_data['access_token'])
    json_response = connect_to_endpoint(url, header)
    return json_response

In [20]:
type(montero_data['tracks']['items'])

list

In [21]:
len(montero_data['tracks']['items'])

15

In [22]:
len(equal_data['tracks']['items'])

14

In [23]:
equal_data['tracks']['items'][0]['id']

'2JiHR4gCDjXwYVRDmwdk95'

In [24]:
montero_track_ids = [x['id'] for x in montero_data['tracks']['items']]
equal_track_ids = [x['id'] for x in equal_data['tracks']['items']]

In [25]:
montero_audio_features = [audio_features(x) for x in montero_track_ids]
equal_audio_features = [audio_features(x) for x in equal_track_ids]

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [26]:
montero_track_features_df = pd.DataFrame(montero_audio_features)
equal_track_features_df = pd.DataFrame(equal_audio_features)

In [27]:
montero_track_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.593,0.503,8,-6.725,0,0.2200,0.29300,0.000000,0.4050,0.710,178.781,audio_features,1SC5rEoYDGUK4NfG82494W,spotify:track:1SC5rEoYDGUK4NfG82494W,https://api.spotify.com/v1/tracks/1SC5rEoYDGUK...,https://api.spotify.com/v1/audio-analysis/1SC5...,137704,4
1,0.685,0.717,0,-6.456,0,0.0411,0.10100,0.000019,0.2660,0.549,127.963,audio_features,3grVoZ10bm2jUGpo7BxpuF,spotify:track:3grVoZ10bm2jUGpo7BxpuF,https://api.spotify.com/v1/tracks/3grVoZ10bm2j...,https://api.spotify.com/v1/audio-analysis/3grV...,221203,4
2,0.741,0.691,10,-7.395,0,0.0672,0.02210,0.000000,0.0476,0.892,150.087,audio_features,5Z9KJZvQzH6PFmb8SNkxuk,spotify:track:5Z9KJZvQzH6PFmb8SNkxuk,https://api.spotify.com/v1/tracks/5Z9KJZvQzH6P...,https://api.spotify.com/v1/audio-analysis/5Z9K...,212353,4
3,0.737,0.846,1,-4.510,0,0.2200,0.00614,0.000000,0.0486,0.546,87.981,audio_features,0e8nrvls4Qqv5Rfa2UhqmO,spotify:track:0e8nrvls4Qqv5Rfa2UhqmO,https://api.spotify.com/v1/tracks/0e8nrvls4Qqv...,https://api.spotify.com/v1/audio-analysis/0e8n...,143901,4
4,0.519,0.464,10,-18.957,0,0.5600,0.90500,0.000000,0.4480,0.287,109.744,audio_features,1BMu5TWvalAGpi5jlhFEBb,spotify:track:1BMu5TWvalAGpi5jlhFEBb,https://api.spotify.com/v1/tracks/1BMu5TWvalAG...,https://api.spotify.com/v1/audio-analysis/1BMu...,24006,4


In [28]:
equal_track_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.631,0.825,11,-3.971,1,0.0433,0.01670,0.000000,0.1220,0.427,129.942,audio_features,2JiHR4gCDjXwYVRDmwdk95,spotify:track:2JiHR4gCDjXwYVRDmwdk95,https://api.spotify.com/v1/tracks/2JiHR4gCDjXw...,https://api.spotify.com/v1/audio-analysis/2JiH...,195760,3
1,0.788,0.859,2,-2.724,1,0.0856,0.28100,0.000000,0.0424,0.822,141.020,audio_features,50nfwKoDiSYg8zOCREWAm5,spotify:track:50nfwKoDiSYg8zOCREWAm5,https://api.spotify.com/v1/tracks/50nfwKoDiSYg...,https://api.spotify.com/v1/audio-analysis/50nf...,207853,4
2,0.587,0.267,8,-9.206,1,0.0349,0.84000,0.000000,0.1210,0.352,98.845,audio_features,5QYnNhTKsN3kE7OaqILA1U,spotify:track:5QYnNhTKsN3kE7OaqILA1U,https://api.spotify.com/v1/tracks/5QYnNhTKsN3k...,https://api.spotify.com/v1/audio-analysis/5QYn...,185853,4
3,0.807,0.893,11,-3.745,0,0.0347,0.04510,0.000028,0.3660,0.537,126.011,audio_features,3rmo8F54jFF8OgYsqTxm5d,spotify:track:3rmo8F54jFF8OgYsqTxm5d,https://api.spotify.com/v1/tracks/3rmo8F54jFF8...,https://api.spotify.com/v1/audio-analysis/3rmo...,230747,4
4,0.520,0.849,9,-3.794,0,0.1860,0.00253,0.000000,0.1190,0.774,175.908,audio_features,4btFHqumCO31GksfuBLLv3,spotify:track:4btFHqumCO31GksfuBLLv3,https://api.spotify.com/v1/tracks/4btFHqumCO31...,https://api.spotify.com/v1/audio-analysis/4btF...,236907,4


In [29]:
montero_track_features_df.to_csv('Montero_Audio_Features.csv')

In [30]:
equal_track_features_df.to_csv('Equal_Audio_Features.csv')

In [31]:
def track_features(track_id):
    url = 'https://api.spotify.com/v1/tracks/{}'.format(track_id)
    header = create_header(session_start_data['access_token'])
    json_response = connect_to_endpoint(url, header)
    return json_response

In [32]:
montero_track_data = [track_features(x) for x in montero_track_ids]
equal_track_data = [track_features(x) for x in equal_track_ids]

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [33]:
montero_track_data_df = pd.DataFrame(montero_track_data)
equal_track_data_df = pd.DataFrame(equal_track_data)

In [34]:
montero_track_data_df.head()

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,137704,True,{'isrc': 'USSM12100531'},{'spotify': 'https://open.spotify.com/track/1S...,https://api.spotify.com/v1/tracks/1SC5rEoYDGUK...,1SC5rEoYDGUK4NfG82494W,False,MONTERO (Call Me By Your Name),91,https://p.scdn.co/mp3-preview/a6e05149543aaa36...,1,track,spotify:track:1SC5rEoYDGUK4NfG82494W
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,221203,True,{'isrc': 'USSM12105731'},{'spotify': 'https://open.spotify.com/track/3g...,https://api.spotify.com/v1/tracks/3grVoZ10bm2j...,3grVoZ10bm2jUGpo7BxpuF,False,DEAD RIGHT NOW,79,https://p.scdn.co/mp3-preview/5183150db97cee38...,2,track,spotify:track:3grVoZ10bm2jUGpo7BxpuF
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,212352,True,{'isrc': 'USSM12104539'},{'spotify': 'https://open.spotify.com/track/5Z...,https://api.spotify.com/v1/tracks/5Z9KJZvQzH6P...,5Z9KJZvQzH6PFmb8SNkxuk,False,INDUSTRY BABY (feat. Jack Harlow),91,https://p.scdn.co/mp3-preview/c1cb40d748692992...,3,track,spotify:track:5Z9KJZvQzH6PFmb8SNkxuk
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,143901,True,{'isrc': 'USSM12105732'},{'spotify': 'https://open.spotify.com/track/0e...,https://api.spotify.com/v1/tracks/0e8nrvls4Qqv...,0e8nrvls4Qqv5Rfa2UhqmO,False,THATS WHAT I WANT,95,https://p.scdn.co/mp3-preview/75aa4d781c96fe99...,4,track,spotify:track:0e8nrvls4Qqv5Rfa2UhqmO
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,24005,False,{'isrc': 'USSM12105733'},{'spotify': 'https://open.spotify.com/track/1B...,https://api.spotify.com/v1/tracks/1BMu5TWvalAG...,1BMu5TWvalAGpi5jlhFEBb,False,THE ART OF REALIZATION,28,https://p.scdn.co/mp3-preview/e86b2ac42600eea6...,5,track,spotify:track:1BMu5TWvalAGpi5jlhFEBb


In [35]:
equal_track_data_df.head()

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,195760,False,{'isrc': 'GBAHS2100670'},{'spotify': 'https://open.spotify.com/track/2J...,https://api.spotify.com/v1/tracks/2JiHR4gCDjXw...,2JiHR4gCDjXwYVRDmwdk95,False,Tides,76,https://p.scdn.co/mp3-preview/50df74b9445911ae...,1,track,spotify:track:2JiHR4gCDjXwYVRDmwdk95
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,207853,False,{'isrc': 'GBAHS2100671'},{'spotify': 'https://open.spotify.com/track/50...,https://api.spotify.com/v1/tracks/50nfwKoDiSYg...,50nfwKoDiSYg8zOCREWAm5,False,Shivers,77,https://p.scdn.co/mp3-preview/bb9f413d6876e265...,2,track,spotify:track:50nfwKoDiSYg8zOCREWAm5
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,185853,False,{'isrc': 'GBAHS2100672'},{'spotify': 'https://open.spotify.com/track/5Q...,https://api.spotify.com/v1/tracks/5QYnNhTKsN3k...,5QYnNhTKsN3kE7OaqILA1U,False,First Times,77,https://p.scdn.co/mp3-preview/30aee656a486465a...,3,track,spotify:track:5QYnNhTKsN3kE7OaqILA1U
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,230746,False,{'isrc': 'GBAHS2100318'},{'spotify': 'https://open.spotify.com/track/3r...,https://api.spotify.com/v1/tracks/3rmo8F54jFF8...,3rmo8F54jFF8OgYsqTxm5d,False,Bad Habits,74,https://p.scdn.co/mp3-preview/e0440ae1a5a38687...,4,track,spotify:track:3rmo8F54jFF8OgYsqTxm5d
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,236906,False,{'isrc': 'GBAHS2100673'},{'spotify': 'https://open.spotify.com/track/4b...,https://api.spotify.com/v1/tracks/4btFHqumCO31...,4btFHqumCO31GksfuBLLv3,False,Overpass Graffiti,82,https://p.scdn.co/mp3-preview/fde3dbd32ba9483e...,5,track,spotify:track:4btFHqumCO31GksfuBLLv3


In [36]:
montero_track_data_df.to_csv('Montero_Track_Data.csv')
equal_track_data_df.to_csv('Equal_Track_Data.csv')